In [1]:
!pip -q install langchain langchain_community langchain-groq tiktoken chromadb huggingface_hub transformers sentence-transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Setting up LangChain


In [3]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from glob import glob
import os
import pandas as pd

## Initialize models and embeddings


In [15]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initializing GROQ chat with provided API key, model name, and settings
model = ChatGroq(
            groq_api_key=groq_api_key, model_name="llama3-70b-8192",
                         temperature=0.2)

## Prompt templates

In [31]:
prompt_template_questions = """
You are a question bank generator for a question answering chatbot app.
Using the provided text below which is a section of a student manual, you are to generate all the possible questions from the students, teachers, parents, and staff that can be asked and answered within the given context.

------------
{text}
------------

Instructions:
Provide a complete list of questions that can be generated based on the context above..
Make sure the answers are exactly from the text (in verbatim) and should not be replaced with any synonyms or added with extra words.
This is for training an extractive question answering model.
Make sure that no parts of the contexts are missed.
Generate at least 30 questions in which the order of the questions match the order of appearance in the context.
Do not include questions that cannot be answered using the context.
Do not repeat any questions or generate the same question again in the list of questions.

Response format for each item:
Q: Question
A: Answer

QUESTIONS AND ANSWERS:
"""

PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["text"],
    template=prompt_template_questions
)

## Load multiple and process documents

In [32]:
# Initialize documents and path
documents = []
root  = os.getcwd()

# Sort the file names
paths = sorted(glob('*.txt'))

# Copy each file to new file
for path in paths:
  loader = TextLoader(path)
  document = loader.load()
  documents.extend(document)

## Loop through each topic and generate questions

In [33]:
# Initialize values
topic = 0
QA_pairs = []

for doc in documents:

  # Increment topic count
  topic += 1

  # Splitting the text into chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
  texts = text_splitter.split_documents([doc])

  # Initialize question generation chain
  question_gen_chain = LLMChain(llm=model, prompt=PROMPT_QUESTIONS)

  # Run question generation chain
  response = question_gen_chain.invoke(texts)

  # Reset count of QA pairs
  pair_count = 0

  # Segregate and store questions and answers
  sentences = response['text'].split('\n')
  for sentence in sentences:
    if sentence.startswith('Q:'):
      Q = sentence[3:]
    elif sentence.startswith('A:'):
      A = sentence[3:]
      QA_pairs.append([Q,A,topic])
      pair_count += 1

  print(f'Topic {topic}: Generated {pair_count} pairs of questions')

Topic 1: Generated 36 pairs of questions
Topic 2: Generated 20 pairs of questions
Topic 3: Generated 30 pairs of questions
Topic 4: Generated 36 pairs of questions
Topic 5: Generated 34 pairs of questions
Topic 6: Generated 29 pairs of questions
Topic 7: Generated 18 pairs of questions
Topic 8: Generated 19 pairs of questions
Topic 9: Generated 11 pairs of questions
Topic 10: Generated 20 pairs of questions
Topic 11: Generated 19 pairs of questions
Topic 12: Generated 26 pairs of questions
Topic 13: Generated 27 pairs of questions
Topic 14: Generated 36 pairs of questions
Topic 15: Generated 21 pairs of questions
Topic 16: Generated 29 pairs of questions
Topic 17: Generated 24 pairs of questions
Topic 18: Generated 18 pairs of questions
Topic 19: Generated 31 pairs of questions
Topic 20: Generated 27 pairs of questions
Topic 21: Generated 25 pairs of questions
Topic 22: Generated 23 pairs of questions
Topic 23: Generated 27 pairs of questions
Topic 24: Generated 28 pairs of questions
T

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hwt7dspheegrqkeh1sgvcdk8` on tokens per minute (TPM): Limit 4500, Used 4675, Requested ~4613. Please try again in 1m3.846666666s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

## Storing the QA pairs

In [34]:
# Create a DataFrame from the list of question-answer pairs
qa_df = pd.DataFrame(QA_pairs, columns=["Question", "Answer", "Topic"])

# Save the DataFrame to a CSV file
qa_df.to_csv("questions_and_answers.csv", index=False)
print('Successfully saved questions and answers to CSV!')

Successfully saved questions and answers to CSV!


## Displaying the QA pairs per topic

In [35]:
# Print sample questions per topic
pd.set_option('display.max_colwidth', None)  # or 199
for i in range(1, len(documents)+1):
  df_topic = qa_df[qa_df['Topic'] == i]
  display(df_topic.head(50))

,Question,Answer,Topic
0,What is the University of San Carlos (USC) administered by?,Society of the Divine Word (SVD) missionaries.,1
1,Since what year has the University of San Carlos (USC) been administered by the Society of the Divine Word (SVD) missionaries?,1935.,1
2,What type of educational institution is the University of San Carlos (USC)?,A Catholic educational institution.,1
3,What is offered by the University of San Carlos (USC) from kindergarten to graduate school?,The complete educational package.,1
4,What is the University of San Carlos (USC) also referred to as?,University.,1
5,When did the University of San Carlos (USC) become a University?,1948.,1
6,Why did the University of San Carlos (USC) expand to the Boys’ High School in 1956?,Due to rapid growth in the ‘50s that saturated the campus near the city center.,1
7,What was the Boys’ High School renamed to?,North Campus.,1
8,When did the University of San Carlos (USC) expand to the Teacher Education Center and Girls’ High School?,1964.,1
9,What was the Teacher Education Center and Girls’ High School renamed to?,South Campus.,1


,Question,Answer,Topic
36,"What is embodied in the two statements ""Education with a Mission"" and ""Witness to the Word""?",The University's Catholic identity.,2
37,What is the aim of USC in terms of the holistic formation of the individual?,"To enlighten the mind, hone skills, and positively transform character and behavior.",2
38,What is the goal of every Carolinian?,"To become a catalyst towards the realization of the kingdom of God, promoting truth, love, justice, freedom, peace, and grace.",2
39,How are Carolinians recognizable in society?,"By their contribution as the ""light"", ""salt"", and ""yeast"" of the Gospel.",2
40,What are the characteristics of USC graduates?,"Professionally competent and skilled (Scientia), having noble character and being value-driven (Virtus), and dedicated to social transformation (Devotio).",2
41,What does the University of San Carlos envision for the world?,A world where the darkness of sin and the night of unbelief vanish before the light of the Word and the Spirit of grace.,2
42,What does the University of San Carlos envision for society?,"A society where citizens are competent, noble in character, and community-oriented.",2
43,What is the University of San Carlos?,A Catholic institution of learning that embodies the principles of academic discipline of San Carlos Borromeo and the missionary charism of the Society of the Divine Word (SVD).,2
44,What is the mission of the University of San Carlos?,"To develop competent and socially responsible professionals and lifelong learners in an environment that fosters excellence in the academic core processes of teaching-learning, research, and community extension service.",2
45,What is the goal of the University's mission?,"To provide timely, relevant, and transformative academic programs responsive to the needs of the local, national, and global communities in a rapidly changing world.",2


,Question,Answer,Topic
56,What is the University seal?,The University seal is the official insignia of the University of San Carlos.,3
57,What does the University seal depict?,The University seal depicts a shield on either side by seven laurel leaves.,3
58,What do the laurel leaves stand for?,The laurel leaves stand for excellence.,3
59,How many gifts of the Holy Spirit do the laurel leaves symbolize?,"Each one symbolizes a gift of the Holy Spirit: wisdom, understanding, counsel, fortitude, knowledge, piety, and fear of the Lord.",3
60,How is the shield divided?,The shield is divided into three parts.,3
61,What is on the upper left of the shield?,On the upper left of the shield is a globe encircled and topped by a cross.,3
62,What does the globe encircled and topped by a cross symbolize?,This is the symbol of the SVD in the world.,3
63,What is on the upper right part of the shield?,On the upper right part of the shield is Magellan’s Cross.,3
64,What does Magellan’s Cross symbolize?,"Magellan’s Cross symbolizes Cebu City, where USC is located.",3
65,What is on the bottom half of the shield?,On the bottom half of the shield is an open book.,3


,Question,Answer,Topic
86,When was the Colegio de San Ildefonso established?,21 August 1595,4
87,Who established the Colegio de San Ildefonso?,"Jesuit priests Antonio Sedeño, Alonso de Humanes, Mateo Sanchez, and a lay brother Gaspar Garay",4
88,Why was the Colegio de San Ildefonso closed in 1769?,Because of the papal expulsion of the Jesuits from Spain and all Spanish possessions in 1767,4
89,Who took over the ownership of the colegio and its properties in 1779?,"The Bishop of Cebu, Msgr. Rubio de Arevalo",4
90,What was the colegio re-established as in 1779?,The Real Seminario de San Carlos,4
91,Who took over the colegio in 1852?,The Dominican priests,4
92,What was the colegio renamed to in 1852?,Seminario Conciliar de San Carlos,4
93,Who took over the colegio in 1867?,The Vincentians,4
94,What was the colegio renamed to in 1867?,Seminario-Colegio de San Carlos,4
95,When did the Seminario-Colegio de San Carlos confer the Bachelor’s degree to its first graduates?,1894,4


,Question,Answer,Topic
122,What was the Talamban campus known as by 1965?,"San Carlos Borromeo (1538-1584), St. Arnold Janssen (1837-1909), St. Joseph Freinademetz (1852-1908).",5
123,What was the Talamban campus known as by 1965?,The Society of Divine Word (SVD).,5
124,What was the Talamban campus known as by 1965?,November 4.,5
125,What was the Talamban campus known as by 1965?,He facilitated the final deliberations of the Council of Trent and took a large share in the drafting of the Tridentine Catechism.,5
126,What was the Talamban campus known as by 1965?,"He bravely attended to the sick and the dead victims of the plague, sparing no expense and avoiding no danger in an effort to assist the poor.",5
127,What was the Talamban campus known as by 1965?,He played a major role against the reformation movement.,5
128,What was the Talamban campus known as by 1965?,"He worked towards the reform of collegiate churches, seminaries, colleges, and communities for the education, especially of candidates for holy orders.",5
129,What was the Talamban campus known as by 1965?,"Because of his series of activities toward reform of the collegiate churches, seminaries, colleges, and communities for the education, especially of candidates for holy orders.",5
130,What was the Talamban campus known as by 1965?,A saint of learning and the arts.,5
131,What was the Talamban campus known as by 1965?,St. Arnold Janssen.,5


...

,Question,Answer,Topic
1461,What is a student organization?,"A student organization is an organized group of students sharing common interests, objectives, philosophy, and operational activities not contrary to laws, morals and University rules and regulations.",57
1462,What is the significance of student organizations in the University?,The University regards the formation of student organizations as a healthy sign of growing social awareness and concern and as an exercise of students’ freedom of association.,57
1463,What are the responsibilities that come with the freedom of association?,The responsibility to respect the freedom of others and to uphold the dignity of the student body as a whole.,57
1464,What are the venues that student organizations provide for students?,"The development of students’ talents, potentials and capabilities; students’ greater awareness of their rights, duties and responsibilities; wholesome social and recreational activities; and the realization of the university’s mission statement and academic aims.",57
1465,How are student organizations categorized?,"Student organizations are broadly categorized into two, namely: Co-curricular and Extra-curricular.",57
1466,What are Co-curricular organizations?,Co-curricular organizations are school/department-based and whose membership is automatic upon enrollment in a particular course or college.,57
1467,What are Extra-curricular organizations?,"Extra-curricular organizations are initiated and organized by students to accommodate specific interests, concerns or visions, e.g. socio-civic, cultural, religious, athletic, professional service clubs, etc. and whose membership is voluntary.",57
1468,Why are student organizations allowed to initiate and organize activities?,To complement and enhance the learning process.,57
1469,Who manages the activities of student organizations?,"Schools, departments, and student organizations, in coordination with the Office of Student Formation and Activities (OSFA), manage these activities.",57
1470,When should student organization activities be held?,"Activities must be held within the academic calendar, and should not be scheduled a week before and during major examinations and institutional celebrations unless the activities are intrinsically related therewith.",57


,Question,Answer,Topic
1480,What is the purpose of out-of-classroom activities in the context of learning?,to augment classroom learning,58
1481,What governs the out-of-classroom activities?,"CMO 63, series of 2017 (Policies and Guidelines of Local Off-Campus Activities)",58
1482,"What is the purpose of CMO 63, series of 2017?",to ensure security and safety of students during the conduct of the activities,58
1483,Where should students file an application for out-of-classroom activities?,at the Office of the Vice President for Academic Affairs,58
1484,Where should student organizations file an application for out-of-classroom activities?,at the Office of Student Formation and Activities,58
1485,Are international trips allowed?,"yes, in some cases",58
1486,What governs international trips?,"CMO 26, series of 2015",58
1487,Are the guidelines subject to updates?,yes,58
1488,Who may provide updates to the guidelines?,CHED,58
1489,What is the title of the topic discussed in this section?,Educational Tours and Field Trips,58


,Question,Answer,Topic
1503,What is the University of San Carlos certified for?,ISO 9001:2015,59
1504,What is the purpose of the quality management system established by the University of San Carlos?,"Provision of student support services for student welfare, student development, and institutional student programs.",59
1505,What is the interactive and popularized version of the quality management system called?,Caloy,59
1506,What can students access through the Caloy kiosks?,Further information regarding the services and programs outlined in the directory.,59
1507,Where are the Caloy kiosks strategically located?,In different buildings in the University.,59
1508,What is the quality policy of the Student Support and Services System of the University of San Carlos?,"In support of academic instruction, the Student Support and Services System of the University of San Carlos is committed to delivering quality services and programs essential in the formation of Carolinian graduates to become productive citizens of the country and the world.",59
1509,What is the commitment of the support units?,"To continuously improve their services, processes, and our Quality Management System for the enhancement of the student experience in the University.",59
1510,"Who may be required to participate in the improvement of services, processes, and the Quality Management System?",Suppliers and other external providers of processes and services.,59
1511,What are the priorities of the Student Support and Services System?,"Achievement of outcomes defined in KRA 4 of CHED CMO 46 relating to student quality, diversity of the population, rate of completion, employment rate, quality of formation and; level of competency, leadership qualities, level of empowerment of graduates.",59
1512,What is the second priority of the Student Support and Services System?,Operation of processes in accordance with the administrative and financial policies of the university.,59


,Question,Answer,Topic


,Question,Answer,Topic
